In [25]:
import torch

In [26]:
emb=torch.load('chunked_emb.pt')
emb=torch.stack(emb)
emb.shape

torch.Size([2681507, 768])

In [27]:
token_chunks=torch.load('clean_data.pt')
token_chunks

[{'doc': 8668, 'count': 135, 'chunk': 1, 'tokens': [3]},
 {'doc': 22706, 'count': 102, 'chunk': 1, 'tokens': [3]},
 {'doc': 76047, 'count': 71, 'chunk': 1, 'tokens': [3]},
 {'doc': 144972, 'count': 81, 'chunk': 1, 'tokens': [3]},
 {'doc': 160187, 'count': 23, 'chunk': 1, 'tokens': [3]},
 {'doc': 169269, 'count': 22, 'chunk': 1, 'tokens': [3]},
 {'doc': 170188, 'count': 40, 'chunk': 1, 'tokens': [3]},
 {'doc': 195773, 'count': 50, 'chunk': 1, 'tokens': [3]},
 {'doc': 218420, 'count': 96, 'chunk': 1, 'tokens': [3]},
 {'doc': 237444, 'count': 21, 'chunk': 1, 'tokens': [3]},
 {'doc': 239593, 'count': 38, 'chunk': 1, 'tokens': [3]},
 {'doc': 246763, 'count': 43, 'chunk': 1, 'tokens': [3]},
 {'doc': 250319, 'count': 69, 'chunk': 1, 'tokens': [3]},
 {'doc': 311720, 'count': 31, 'chunk': 1, 'tokens': [3]},
 {'doc': 351617, 'count': 24, 'chunk': 1, 'tokens': [3]},
 {'doc': 351953, 'count': 261, 'chunk': 1, 'tokens': [3]},
 {'doc': 358198, 'count': 18, 'chunk': 1, 'tokens': [3]},
 {'doc': 375809

In [28]:
import faiss

In [5]:
index = faiss.IndexFlatL2(emb.shape[1])
index.add(emb)

In [7]:
from transformers import AutoTokenizer, BertModel
model_name="avichr/Legal-heBERT"

In [9]:
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=BertModel.from_pretrained(model_name)

Some weights of BertModel were not initialized from the model checkpoint at avichr/Legal-heBERT and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
import datasets
data = datasets.load_dataset('LevMuchnik/SupremeCourtOfIsrael')
texts=data['train']['text']

Found cached dataset parquet (/home/user/.cache/huggingface/datasets/LevMuchnik___parquet/LevMuchnik--SupremeCourtOfIsrael-7f46ab7220db2581/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
#test_text="עירעור על מעצר"
#test_text="תיק מנהלי"
#test_text="מחלת הקורונה מפריעה לעובדים לעבוד"
test_text="שוחד, מרמה והפרת אמונים"
#test_text="שוחד של נבחרי ציבור"
#test_text="עצירים מנהליים מהפרות סדר ביהודה ושומרון"
#test_text="תנובה לשיווק תוצרת חקלאית"
#test_text="ראש ממשלה מושחת"
# test_text="ראש ממשלה מושחט"
#test_text="טייקון מרמה את הציבור על ידי שיחוד עדי מדינה"
#test_text="תיקון לחוק אשר מאפשר חופש ביטוי אומנותי " #didnt work
#test_text="איך בית המשפט העליון פסק בעבר בנושאים של זכויות פרט ופרטיות במקרים שבהם הייתה עימות בין זכויות אלו לבין צורכי הביטחון הלאומי, ומה היו השיקולים העיקריים שהנחו את השופטים בקביעת הגבולות והמאזן בין הזכויות האלו?"


In [34]:
with torch.no_grad():
    inputs=tokenizer(test_text,return_tensors='pt',truncation=True,padding=True,max_length=model.config.max_position_embeddings)
    inputs={k:v.to(model.device) for k,v in inputs.items()}
    key=model(**inputs).last_hidden_state.mean([1]).cpu()

In [35]:
_,idx=index.search(key,3)
idx

array([[2115430, 2379837, 2234385]])

In [36]:
ans=[tokenizer.decode(token_chunks[i]['tokens']) for i in idx[0]]
ans

['[CLS] המשיב ( בערעור המדינה ) והמערער ( בערעורו שלו ) הורשע בביצוע שורה ארוכה של עבירות מרמה והפרת אמונים, זיוף בנסיבות מחמירות, גניבה בידי מורשה, הלבנת הון, גניבה בידי עובד ציבור וגניבה בידי מנהל, מרמה והפרת אמונים בתאגיד, רישום כוזב במסמכי תאגיד וקבלת דבר במרמה בנסיבות מחמירות. [SEP]',
 '[CLS] 2. נגד העוררת הוגש כתב אישום ב - 13 אישומים, בעבירות רבות חוזרות ונשנות של התחזות כאדם אחר, קבלת דבר במרמה בנסיבות מחמירות, זיוף מסמך בכוונה לקבל באמצעותו דבר, שימוש במסמך מזוייף, שיבוש מהלכי משפט, הפרת הוראה חוקית, קבלת דבר במרמה, איסור הלבנת הון, נהיגה בזמן פסילה, התחזות לרופא, שימוש בכינוי " רופא ", חבלה חמורה, תקיפה, תקיפה הגורמת חבלה של ממש, מעשה פזיזות ורשלנות, הונאה בכרטיס חיוב, מרמה, מרמה או תחבולה, אי - קיום דרישות ואי - הוצאת חשבונית מס. [SEP]',
 '[CLS] העורר שימש כמפקד שירותי הכבאות וההצלה באזור טבריה. בתאריך 23. 7. 08 הגישה המשיבה נגדו כתב - אישום לבית המשפט המחוזי בנצרת, בו יוחסו לו עבירות של גניבה בידי מנהל, גניבה בידי עובד ציבור, קבלת דבר במרמה בנסיבות מחמירות, מרמה והפרת אימונ

In [21]:
import re

In [24]:
ans=[re.sub(r'\n\s*\n\s*\n+', '\n\n', texts[token_chunks[i]['doc']]) for i in idx[0]]
print(ans[0])



בבית המשפט העליון

בג"ץ 
  8558/04

בפני:   

כבוד השופטת א' פרוקצ'יה

העותר:

ב.ל סלס שירותי מזון בע"מ

נ
  
ג
  
ד

המשיבים:

1. ראש הממשלה
  והממונה על ענייני  דתות - אריאל שר

2. מועצת הרבנות הראשית לישראל

3. הרב הראשי לבאר שבע - הרב דרעי יהודה

4. היועץ המשפטי לממשלה

עתירה למתן צו על תנאי

החלטה

           העתירה
תועבר לדיון בפני הרכב.

           אין מקום למתן צו ביניים.

           ניתנה היום, כ"ב בסיון תשס"ה
(29.6.05).

                                                                                      ש
ו פ ט ת

_________________________

העותק כפוף לשינויי עריכה וניסוח.

04085580_R02.doc

מרכז מידע, טל' 02-6750444 ; אתר אינטרנט,  
www.court.gov.il




# make test data

In [46]:
import os
from os.path import join

In [47]:
sample_idx=[token_chunks[i]['doc'] for i in idx[0]]+list(range(100))
sample_idx[:7]

[411839, 511505, 322708, 0, 1, 2, 3]

In [49]:
#os.mkdir('test_data')

In [51]:
for i in sample_idx:
    with open(join('test_data',f'{i}.txt'),'w') as f:
        f.write(texts[i])